In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    data['DOLocationID'] = data['DOLocationID'].astype(str)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [8]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [9]:
def get_weights(rawdata, zontoBorough):
    
    rawdata['Borough'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
    
    borough_df = rawdata[['vehicle_count','Borough']].groupby(by='Borough').sum().reset_index()

    zone_df = rawdata[['vehicle_count','DOLocationID']].groupby(by='DOLocationID').sum().reset_index()

    zone_df['Borough'] = zone_df['DOLocationID'].apply(lambda x:zontoBorough[x])

    zone_df = pd.merge(borough_df, zone_df, on=['Borough'], how='inner')

    zone_df['zone_weight'] = zone_df.vehicle_count_y / zone_df.vehicle_count_x

    zone_df = zone_df[['Borough', 'DOLocationID', 'zone_weight']]

    return zone_df

#### Load Raw Data

In [10]:
hub = 'EWR'
tune_hyp_params = False

In [11]:
dataDir = '/Users/hemingyi/Documents/UrbanTemporalNetworks/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [12]:
rawdata = loadData(file)

Raw shape:  (2260080, 4)
Days:  365


In [13]:
rawdata.head(2)

,DOLocationID,Date,Hour,vehicle_count
0,2,2018-01-01,0,0.0
1,3,2018-01-01,0,0.0


In [14]:
edge_data = getTimeSeries(rawdata)
edge_data = edge_data.reset_index()
edge_data.head(3)

DOLocationID,Date,Hour,10,100,101,102,106,107,108,109,...,90,91,92,93,94,95,96,97,98,99
0,2018-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Burrough Level Aggregation

In [16]:
zones = pd.read_csv('/Users/hemingyi/Documents/UrbanTemporalNetworks/Data/taxi_zones.csv')
zones['LocationID'] = zones['LocationID'].astype(str)
zones.head(2)

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens


In [17]:
zontoBorough = dict(zip(zones.LocationID.values,zones.borough.values))

In [18]:
zone_weights = get_weights(rawdata, zontoBorough)
zone_weights.head(2)

,Borough,DOLocationID,zone_weight
0,Bronx,119,0.011173
1,Bronx,126,0.011173


In [19]:
rawdata['DOLocationID'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
rawdata.head(2)

,DOLocationID,Date,Hour,vehicle_count,Borough
0,Queens,2018-01-01,0,0.0,Queens
1,Bronx,2018-01-01,0,0.0,Bronx


In [20]:
borough_data = getTimeSeries(rawdata)
borough_data = borough_data.reset_index()
borough_data.head(3)

DOLocationID,Date,Hour,Bronx,Brooklyn,Manhattan,Queens,Staten Island
0,2018-01-01,0,0,1,3,0,0
1,2018-01-01,1,0,1,0,0,0
2,2018-01-01,2,0,0,0,0,0


### Merge External Data Features

In [21]:
externalDataDir = "/Users/hemingyi/Documents/UrbanTemporalNetworks/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [22]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,ifsat,...,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow,arrival
0,18/1/1 0:00,263,174,437,1,0,0,0,0,0,...,7,12.5,-21.2,52,0,0.0,0.0,0,0,24.0
1,18/1/1 1:00,138,133,271,1,0,0,0,0,0,...,7,12.5,-21.2,52,0,0.0,0.0,0,0,9.0


In [23]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,ifsat,...,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow,arrival
0,2018-01-01 00:00:00,263,174,437,1,0,0,0,0,0,...,7,12.5,-21.2,52,0,0.0,0.0,0,0,24.0
1,2018-01-01 01:00:00,138,133,271,1,0,0,0,0,0,...,7,12.5,-21.2,52,0,0.0,0.0,0,0,9.0


In [24]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [25]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [26]:
extDf.columns

Index(['date', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue', 'ifwed', 'ifthu',
       'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3', 'if4', 'if5',
       'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12', 'if13', 'if14',
       'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21', 'if22', 'if23',
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow', 'arrival', 'Hour',
       'Dow', 'Date'],
      dtype='object')

In [27]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [28]:
extDf = extDf[selected_columns]

In [29]:
print(borough_data.shape)
print(extDf.shape)

(8760, 7)
(8760, 14)


In [30]:
borough_data['Date'] = pd.to_datetime(borough_data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [31]:
borough_data = pd.merge(borough_data,extDf, on=['Date', 'Hour'], how='inner')
print(borough_data.shape)
borough_data['Date'] = borough_data['Date'].dt.date
borough_data.head()

(8760, 19)


,Date,Hour,Bronx,Brooklyn,Manhattan,Queens,Staten Island,Dow,arrival,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,0,1,3,0,0,0,24.0,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
1,2018-01-01,1,0,1,0,0,0,0,9.0,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
2,2018-01-01,2,0,0,0,0,0,0,3.0,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
3,2018-01-01,3,0,0,0,0,0,0,0.0,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
4,2018-01-01,4,0,0,0,0,0,0,3.0,18,7,12.5,-21.2,52,0,0.0,0.0,0,0


In [32]:
borough_data.columns

Index(['Date', 'Hour', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island', 'Dow', 'arrival', 'maxtemp', 'mintemp', 'avgtemp',
       'departure', 'hdd', 'cdd', 'participation', 'newsnow', 'snowdepth',
       'ifSnow'],
      dtype='object')

In [33]:
lagColumns = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island', 'arrival']

DateColumns = ['Date']

targetColumns = ['Bronx', 'Brooklyn','Manhattan', 'Queens',
       'Staten Island']

In [34]:
maxlag = 12

borough_data_lag = addLag(borough_data, maxlag, lagColumns)

borough_data_lag.shape

(8748, 91)

In [35]:
CommR2List = []
EdgeR2List = []
residualDf_list = []

for m in range(1,13):
    print()
    print("month: ",m)
    month_index  = pd.to_datetime(borough_data_lag.Date).dt.month == m

    dataset_train = borough_data_lag[~month_index]
    dataset_test = borough_data_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)

    edgeMonthIndex = [False] * maxlag + list(month_index)
    edge_testData = edge_data[edgeMonthIndex]
    select_cols = [c for c in edge_testData.columns if c not in ['Date','Hour']]
    edge_testData = edge_testData[select_cols]
    print("edge test data shape: ",edge_testData.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]

    rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                               min_samples_split=3,
                               min_samples_leaf= 2, 
                               max_features= 'sqrt',
                               max_depth= None, 
                               bootstrap= False)

    rf2.fit(X_train,y_train)

    print("Train R2: ",rf2.score(X_train,y_train))
    test_r2 = rf2.score(X_test,y_test)
    print("Test R2: ",test_r2)


    comm_prediction = rf2.predict(X_test)
    edge_prediction_df = pd.DataFrame(comm_prediction)
    edge_prediction_df.columns = y_test.columns

    residual = y_test - comm_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    boroughs = list(edge_prediction_df.columns)
    for bor in boroughs:
    #     print(bor)

        weight_df = zone_weights[zone_weights.Borough == bor]

    #     print(len(weight_df.DOLocationID))

        for b_zone,z_weight in zip(weight_df.DOLocationID.values,weight_df.zone_weight.values):        
            edge_prediction_df[b_zone] = edge_prediction_df[bor] * z_weight


    select_cols = [c for c in edge_prediction_df.columns if c not in boroughs]
    edge_prediction_df = edge_prediction_df[select_cols]


    edge_prediction_df = edge_prediction_df[edge_testData.columns]

    edge_r2 = r2_score(edge_testData.values, edge_prediction_df.values, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)

    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)


month:  1
Train Size:  (8016, 91)
Test Size:  (732, 91)
edge test data shape:  (732, 258)
Train R2:  0.959665043567821
Test R2:  0.49636846040030486
Edge R2:  0.017610822307271772

month:  2
Train Size:  (8076, 91)
Test Size:  (672, 91)
edge test data shape:  (672, 258)
Train R2:  0.9603020220329999
Test R2:  0.4266821519789716
Edge R2:  0.013838007837019058

month:  3
Train Size:  (8004, 91)
Test Size:  (744, 91)
edge test data shape:  (744, 258)
Train R2:  0.9609146539497223
Test R2:  0.4474277618534738
Edge R2:  0.018298346216426738

month:  4
Train Size:  (8028, 91)
Test Size:  (720, 91)
edge test data shape:  (720, 258)
Train R2:  0.9603433915906944
Test R2:  0.4792117848913489
Edge R2:  0.022151338247123998

month:  5
Train Size:  (8004, 91)
Test Size:  (744, 91)
edge test data shape:  (744, 258)
Train R2:  0.9604894198653906
Test R2:  0.4442068747115485
Edge R2:  0.021650521154025885

month:  6
Train Size:  (8028, 91)
Test Size:  (720, 91)
edge test data shape:  (720, 258)
Trai

In [36]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.431427460120281
0.017994065092043546


In [37]:
res_df = pd.concat(residualDf_list, axis = 0)
print(res_df.shape)
res_df.head()

(8748, 7)


,Date,Hour,Bronx,Brooklyn,Manhattan,Queens,Staten Island
12,2018-01-01,12,-0.013333,1.602222,0.113333,-0.084444,-0.004444
13,2018-01-01,13,-0.027778,-0.421667,3.875000,-0.102222,-0.022222
14,2018-01-01,14,-0.033556,3.371000,8.494889,-0.096667,-0.018444
15,2018-01-01,15,-0.060556,0.087889,1.009444,-0.123556,-0.018333
16,2018-01-01,16,-0.058889,2.089444,4.925000,-0.140000,-0.032222


In [38]:
res_df.to_csv('/Users/hemingyi/Documents/UrbanTemporalNetworks/Resid/%sBoroRFCV.csv'%hub.upper())